In [1]:
import gc
import os
import glob
import fitsio
import glob
import catutil
import numpy as np

In [2]:
if False:
    # write the photoz best for selection
    obsDir='/hildafs/projects/phy200017p/share/HSC_shape_catalog/'
    zdat=[]
    for fieldname in catutil.field_names:
        fname= os.path.join(obsDir,'%s_calibrated_pz.fits' %fieldname)
        tmp=fitsio.read(fname)[['object_id','ephor_photoz_best']]
        zdat.append(tmp)
        del tmp
    gc.collect()
    zdat=np.hstack(zdat)
    zdat.sort(order='object_id')
    fitsio.write('/hildafs/projects/phy200017p/share/raytracesim/HSC_S16A/ephor_zbest.fits',zdat)
else:
    zdat=fitsio.read('/hildafs/projects/phy200017p/share/raytracesim/HSC_S16A/ephor_zbest.fits')

In [3]:
def prepare_data(mockDir,isim,irot):
    dall=[]
    for fieldname in catutil.field_names:
        dfield=[]
        sDir= os.path.join(mockDir,fieldname,'r%03d'%isim, 'rotmat%d' %irot,'*.fits')
        fnamelist=glob.glob(sDir)
        for fname in fnamelist:
            data=fitsio.read(fname)
            dfield.append(data)
            del data
            #os.remove(fname)
        del fnamelist,sDir
        dall.append(np.hstack(dfield))
        del dfield
        gc.collect()
    out=np.hstack(dall)
    out.sort(order='object_id')
    return out

In [ ]:
zdat=fitsio.read('/hildafs/projects/phy200017p/share/raytracesim/HSC_S16A/ephor_zbest.fits')
odir='/hildafs/projects/phy200017p/share/raytracesim/HSC_S16A/mocks_4zbins/'
dDir='/hildafs/projects/phy200017p/share/raytracesim/HSC_S16A/downloads/'
for isim in range(25,108):
    print(isim)
    for irot in range(21):
        oname=os.path.join(odir,'r%03d_rot%02d_zbin*.fits' %(isim,irot))
        if len(glob.glob(oname))==4:
            continue
        print(irot)
        mock=prepare_data(dDir,isim,irot)
        assert np.all(mock['object_id']==zdat['object_id'])
        for iz in range(4):
            oname=os.path.join(odir,'r%03d_rot%02d_zbin%d.fits' %(isim,irot,iz+1))
            if os.path.isfile(oname):
                continue
            zmin=(iz+1)*0.3
            zmax=(iz+2)*0.3
            msk=(zdat['ephor_photoz_best']>=zmin)&(zdat['ephor_photoz_best']<zmax)
            fitsio.write(oname,mock[msk])
            del zmin,zmax,msk,oname
        del mock

In [5]:
26*21

546

In [6]:
108*21

2268